In [1]:
# Preliminary imports and auxiliary-function definition

from pathlib import Path
from devtools import debug
import tempfile

from fractal_server.app.runner.v2.models import Dataset, WorkflowTask
from fractal_server.app.runner.v2.runner import execute_tasks_v2
from fractal_server.images import SingleImage

from concurrent.futures import ThreadPoolExecutor
from image_data_exist_on_disk import image_data_exist_on_disk

import sys
sys.path.append("../tests/v2/04_runner")
from fractal_tasks_core_mock import TASK_LIST

executor = ThreadPoolExecutor()

{'name': 'create_ome_zarr_compound', 'input_types': {}, 'output_types': {}, 'function_non_parallel': <cyfunction create_ome_zarr at 0x7653e810a810>, 'function_parallel': <cyfunction yokogawa_to_zarr at 0x7653e810bac0>}
{'name': 'create_ome_zarr_multiplex_compound', 'input_types': {}, 'output_types': {}, 'function_non_parallel': <cyfunction create_ome_zarr_multiplex at 0x7653e810b100>, 'function_parallel': <cyfunction yokogawa_to_zarr at 0x7653e810bac0>}
{'name': 'MIP_compound', 'input_types': {'3D': True}, 'output_types': {'3D': False}, 'function_non_parallel': <cyfunction new_ome_zarr at 0x7653e8151d80>, 'function_parallel': <cyfunction maximum_intensity_projection at 0x7653e8152740>}
{'name': 'illumination_correction', 'input_types': {'illumination_correction': False}, 'output_types': {'illumination_correction': True}, 'function_non_parallel': None, 'function_parallel': <cyfunction illumination_correction at 0x7653e8150380>}
{'name': 'illumination_correction_compound', 'input_types':

In [2]:
# Create temporary directory for mocked zarrs
tmp_path = Path(tempfile.mkdtemp())
zarr_dir = (tmp_path / "zarr_dir").as_posix().rstrip("/")
print(f"{zarr_dir=}")

zarr_dir='/tmp/tmpqqtcb0ro/zarr_dir'


In [3]:
# Run create-ome-zarr-multiplex
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["create_ome_zarr_multiplex_compound"],
            args_non_parallel=dict(image_dir="/tmp/input_images"),
        )
    ],
    dataset=Dataset(zarr_dir=zarr_dir),
    executor=executor,
)

# Print current dataset information
debug(dataset)

# We have 6 images (two wells, three cycles)
assert len(dataset.images) == 6
assert dataset.image_paths == [
    f"{zarr_dir}/my_plate.zarr/A/01/0",
    f"{zarr_dir}/my_plate.zarr/A/01/1",
    f"{zarr_dir}/my_plate.zarr/A/01/2",
    f"{zarr_dir}/my_plate.zarr/A/02/0",
    f"{zarr_dir}/my_plate.zarr/A/02/1",
    f"{zarr_dir}/my_plate.zarr/A/02/2",
]

# Image data do exist on disk
assert image_data_exist_on_disk(dataset.images)

[create_ome_zarr_multiplex] START
[create_ome_zarr_multiplex] image_dir='/tmp/input_images'
[create_ome_zarr_multiplex] zarr_dir='/tmp/tmpqqtcb0ro/zarr_dir'
[create_ome_zarr_multiplex] zarr_path='/tmp/tmpqqtcb0ro/zarr_dir/my_plate.zarr'
[create_ome_zarr_multiplex] END
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmpqqtcb0ro/zarr_dir/my_plate.zarr/A/01/0'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_01_0.tif'
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmpqqtcb0ro/zarr_dir/my_plate.zarr/A/01/1'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_01_1.tif'
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmpqqtcb0ro/zarr_dir/my_plate.zarr/A/01/2'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_01_2.tif'
[yokogawa_to_zarr] END
[yokogawa_to_zarr] END
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmpqqtcb0ro/zarr_dir/my_plate.zarr/A/02/0'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_02_0.tif'
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tm

In [4]:
# Run registration part 1
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["calculate_registration_compound"],
            args_non_parallel=dict(ref_acquisition=0),
        )
    ],
    dataset=dataset,
    executor=executor,
)

# Print current dataset information
debug(dataset)

# The number of images has not changed
assert len(dataset.images) == 6

# Image data do exist on disk
assert image_data_exist_on_disk(dataset.images)

# Images have not changed
for image in dataset.images:
    assert "registration" not in image.types.keys()

/tmp/ipykernel_92894/90610181.py:14 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_multiplex_compound',
            'calculate_registration_compound',
        ],
        zarr_dir='/tmp/tmpqqtcb0ro/zarr_dir',
        images=[
            SingleImage(
                path='/tmp/tmpqqtcb0ro/zarr_dir/my_plate.zarr/A/01/0',
                origin=None,
                attributes={
                    'well': 'A01',
                    'plate': 'my_plate.zarr',
                    'acquisition': 0,
                },
                types={
                    '3D': True,
                },
            ),
            SingleImage(
                path='/tmp/tmpqqtcb0ro/zarr_dir/my_plate.zarr/A/01/1',
                origin=None,
                attributes={
                    'well': 'A01',
                    'plate': 'my_plate.zarr',
                    'acquisition': 1,
                },
                types={
                    '3D': Tru

In [5]:
# Find registration consensus
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["find_registration_consensus"],
        )
    ],
    dataset=dataset,
    executor=executor,
)

# Print current dataset information
debug(dataset)

# The number of images has not changed
assert len(dataset.images) == 6

# Image data do exist on disk
assert image_data_exist_on_disk(dataset.images)

# Images have not changed
for image in dataset.images:
    assert image.attributes.get("registration") is None

/tmp/ipykernel_92894/3129502953.py:13 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_multiplex_compound',
            'calculate_registration_compound',
            'find_registration_consensus',
        ],
        zarr_dir='/tmp/tmpqqtcb0ro/zarr_dir',
        images=[
            SingleImage(
                path='/tmp/tmpqqtcb0ro/zarr_dir/my_plate.zarr/A/01/0',
                origin=None,
                attributes={
                    'well': 'A01',
                    'plate': 'my_plate.zarr',
                    'acquisition': 0,
                },
                types={
                    '3D': True,
                },
            ),
            SingleImage(
                path='/tmp/tmpqqtcb0ro/zarr_dir/my_plate.zarr/A/01/1',
                origin=None,
                attributes={
                    'well': 'A01',
                    'plate': 'my_plate.zarr',
                    'acquisition': 1,
                },
        

In [6]:
# Apply registration to image
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["apply_registration_to_image"],
            args_parallel=dict(overwrite_input=True),
        )
    ],
    dataset=dataset,
    executor=executor,
)

# Print current dataset information
debug(dataset)

# The number of images has not changed
assert len(dataset.images) == 6

# Image data do exist on disk
assert image_data_exist_on_disk(dataset.images)

# Images have been overwritten, and then they are now marked with registration=True
for image in dataset.images:
    assert image.types["registration"] is True

[registration] START
[registration] path='/tmp/tmpqqtcb0ro/zarr_dir/my_plate.zarr/A/01/2'
[registration] overwrite_input=True
[registration] Reading information from /tmp/tmpqqtcb0ro/zarr_dir/my_plate.zarr/A/01/2/registration_table_final
[registration] START
[registration] path='/tmp/tmpqqtcb0ro/zarr_dir/my_plate.zarr/A/02/2'
[registration] overwrite_input=True
[registration] Reading information from /tmp/tmpqqtcb0ro/zarr_dir/my_plate.zarr/A/02/2/registration_table_final
[registration] out={'image_list_updates': [{'path': '/tmp/tmpqqtcb0ro/zarr_dir/my_plate.zarr/A/01/2'}]}
[registration] END
[registration] out={'image_list_updates': [{'path': '/tmp/tmpqqtcb0ro/zarr_dir/my_plate.zarr/A/02/2'}]}
[registration] END
/tmp/ipykernel_92894/1100781836.py:14 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_multiplex_compound',
            'calculate_registration_compound',
            'find_registration_consensus',
            'apply_registration_to

In [7]:
executor.shutdown()